## Model Selection and Training

Once a model is selected,it needs to be trained before applied to real world solutions.
Data is first split into training and testing dataset used to evaluate performance of model, which is then exposed to the test data.  

![data_training](link)
Dataset can be chosen through serial splitting or random selection (80% to 20%)
Here we demonstrate how to select a particular model and traini it.

In [2]:
pip install sklearn

     |████████████████████████████████| 8.0 MB 1.1 MB/s 
     |████████████████████████████████| 306 kB 990 kB/s 
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1316 sha256=bc5243d897be9552b846d1ab67e5e98ce270595ce8ec9fcbcff6bce2d7b622b5
  Stored in directory: /Users/mikemwanga/Library/Caches/pip/wheels/e4/7b/98/b6466d71b8d738a0c547008b9eb39bf8676d1ff6ca4b22af1c
Successfully built sklearn
Note: you may need to restart the kernel to use updated packages.


In [1]:
#Import  libraries for use
#analysis
import pandas as pd
import numpy as np
import random as rnd
import seaborn as sns

#visualization
import matplotlib.pyplot as plt
%matplotlib inline 
%pylab inline

#ML
from sklearn.linear_model import LinearRegression
from sklern.linear_model import LogisticRergression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecissionTree
from sklearn.naive_bayes import GaussianNB

from sklearn.metrics import accuracy_score # for accuracy score
from sklearn.model_selection import kFold #  for cross validation
from sklearn.model_selection import cross_val_score #for evaluating cross validation score
from sklearn.model_selection import cross_val_predict # for prediction
from sklearn.metrics import confusion_matrix # for confussion



Populating the interactive namespace from numpy and matplotlib


ModuleNotFoundError: No module named 'sklern'